In [1]:
import pyximport; pyximport.install()
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import set_types
import monte_carlo

%matplotlib inline

# Reproduce Tristan Barnett Results

## Standard scoring systems

In [2]:
spw_1, spw_2 = 0.77, 0.73

final_set_no_tb = lambda win_probs: set_types.play_standard_set(win_probs, has_tiebreak=False)
final_set_with_tb = lambda win_probs: set_types.play_standard_set(win_probs, has_tiebreak=True)

system_4 = lambda: set_types.best_of(
    set_types.play_standard_set, [spw_1, spw_2], 
    5, final_set_fun=final_set_no_tb)

system_5 = lambda: set_types.best_of(
    set_types.play_standard_set, [spw_1, spw_2], 
    3, final_set_fun=final_set_no_tb)

system_6 = lambda: set_types.best_of(
    set_types.play_standard_set, [spw_1, spw_2], 
    5, final_set_fun=final_set_with_tb)

system_7 = lambda: set_types.best_of(
    set_types.play_standard_set, [spw_1, spw_2], 
    3, final_set_fun=final_set_with_tb)

system_8 = lambda: set_types.best_of(
    set_types.play_standard_set, [spw_1, spw_2], 
    3, final_set_fun=set_types.super_tb)

system_9 = lambda: set_types.best_of(
    lambda win_probs: 
    set_types.play_standard_set(win_probs, service_game_ad=False), 
    [spw_1, spw_2], 
    3, final_set_fun=set_types.super_tb)

all_systems = {4: system_4, 5: system_5, 6: system_6, 7: system_7,
               8: system_8, 9: system_9}

In [5]:
num_trials = 10000

system_results = dict()

for num, fn in all_systems.items():
    
    num_pts = list()
    p1_won = list()
    
    for i in range(num_trials):
        
        result = fn()
        
        num_pts.append(result.total_points)
        
        # Change if we vary who serves first
        p1_won.append(result.final_score[0] > result.final_score[1])
        
    better_won = np.mean(p1_won)
    mean_pts = np.mean(num_pts)
    std_pts = np.std(num_pts)
    pct_98 = np.percentile(num_pts, 98)
        
    system_results[num] = {'better_won': better_won,
                           'mean_pts': mean_pts,
                           'std_pts': std_pts,
                           '98th': pct_98}

In [6]:
pd.DataFrame(system_results)

,4,5,6,7,8,9
98th,569.020000,475.040000,386.000000,242.000000,185.000000,173.000000
better_won,0.718400,0.674900,0.709100,0.671100,0.655700,0.655800
mean_pts,289.264300,192.742900,273.369100,166.457300,142.466100,131.204300
std_pts,97.224738,93.332216,60.986727,40.187154,21.820033,20.855425
